In [6]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
# from src.load_transform_data import get_new_ethereum_ohlc, get_new_ethereum_ohlc_2
from src.hopsworks_connections import pull_data, upload_data, pull_model
import pandas_ta as ta
import requests
from datetime import datetime, timedelta
import time

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


## Earlier, I used to scrape data from coinlore.com, but I realized they update it several hours late every day, so now I use coinGecko API instead. This was my code for that...

### Download the last 30 days of Ethereum OHLC data from coinlore.com

In [2]:
##### This is the scraping function that is used in github actions - it wont work locally unless you have chromeDriver installed #####
# But if you want ChromeDriver locally, you can do this (Mac):
# 1. brew install chromedriver
# 2. xattr -d com.apple.quarantine $(which chromedriver)
# 3. which chromedriver
# 4. the above line give the path to chromedriver. Now you can go into src/load_transfrom_data.py, and change the path in the get_new_ethereum_ohlc() function.  
# new_eth_ohlc = get_new_ethereum_ohlc()

##### Use this function when you running locally. This one doesn't require chromeDriver
# new_eth_ohlc = get_new_ethereum_ohlc_2()

# new_eth_ohlc.head()

### clean the new raw ethereum data so hopsworks accepts it

In [3]:
# def convert_value(value):
#     """
#     Converts a string value to a float. Removes $ signs, and converts
#     billion (bn), million (m), and thousand (K) values to their numeric equivalents.
#     """
#     value = value.replace('$', '')  # Remove $ sign to simplify processing
#     if value[-1].lower() == 'm':
#         return float(value[:-1]) * 1_000_000
#     elif value[-1].lower() == 'b':
#         return float(value[:-1]) * 1_000_000_000
#     elif value[-1].lower() == 'k':
#         return float(value[:-1]) * 1_000
#     elif value[-2:].lower() == 'bn':  # Handle 'bn' for billions
#         return float(value[:-2]) * 1_000_000_000
#     else:
#         return float(value)

# new_eth_ohlc.columns = [col.lower() for col in new_eth_ohlc.columns]
# new_eth_ohlc.rename(columns={'volume(eth)': 'volume_eth', 'market cap': 'market_cap'}, inplace=True)

# for col in ['open', 'high', 'low', 'close', 'volume', 'market_cap']:
#     new_eth_ohlc[col] = new_eth_ohlc[col].apply(convert_value)

# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date'])

# new_eth_ohlc.head()

### Combine this latest raw Etherum data with existing raw Ethereum data in hopsworks, and save it back into hopsworks

In [4]:
# raw_eth_ohlc = pull_data('raw_ethereum_ohlc', 1, 'raw_ethereum_ohlc_view', 1)
# raw_eth_ohlc

In [5]:
# # Hopsowrks returns fucked date column object, with weird time zone formats, we need to standardize first
# raw_eth_ohlc['date'] = pd.to_datetime(raw_eth_ohlc['date']).dt.tz_localize(None)
# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date']).dt.tz_localize(None)

# combined_raw_eth_ohlc = pd.concat([new_eth_ohlc, raw_eth_ohlc], ignore_index=True)
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.drop_duplicates(subset='date', keep='last')
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.sort_values(by='date')
# combined_raw_eth_ohlc

In [6]:
# upload_data(combined_raw_eth_ohlc, 'raw_ethereum_ohlc', 1, 'Raw ethereum OHLC (open, high, low, close) data from coinlore.com')

## Get recent data using CoinGecko API

In [2]:
datetime.now()

datetime.datetime(2024, 4, 22, 23, 15, 36, 712892)

In [3]:
def fetch_recent_data(crypto):
    # 20:00, or 8pm in EST is 12:00 in UTC, which is the cycle by which crypto OHLC are defined
    end_date = datetime.now().replace(hour=20, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=1)

    url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': int(start_date.timestamp()),
        'to': int(end_date.timestamp())
    }
    
    response = safe_request(url, params)
    
    if response and response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['date', 'price'])
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['market_cap'] = pd.DataFrame(data['market_caps'])[1].values
        df['volume_24h'] = pd.DataFrame(data['total_volumes'])[1].values
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}" if response else "Failed to fetch data; no response.")
    
    # Include only the last day
    df['date'] = pd.to_datetime(df['date'])
    # Step 2: Extract the date part from the 'datetime' column
    df['no_hour_date'] = df['date'].dt.date
    earliest_date = df['no_hour_date'].min()
    filtered_df = df[df['no_hour_date'] == earliest_date]
    filtered_df = filtered_df.drop(columns='no_hour_date')
    
    return filtered_df

def safe_request(url, params, retries=5, backoff_factor=0.5):
    for i in range(retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                # We're being rate-limited; back off and retry
                sleep_time = backoff_factor * (2 ** i)
                print(f"Rate limit hit. Waiting {sleep_time:.2f} seconds before retrying...")
                time.sleep(sleep_time)
            else:
                # Other errors, break the retry loop and return None
                print(f"Request failed with status code {response.status_code}.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Retrying...")
            time.sleep(backoff_factor * (2 ** i))

    return None

In [6]:
eth_raw = fetch_recent_data("ethereum")
eth_raw

,date,price,market_cap,volume_24h
0,2024-04-23 00:01:40.444,3200.817053,3.844669e+11,1.192234e+10
1,2024-04-23 00:05:51.581,3198.421920,3.844669e+11,1.225807e+10
2,2024-04-23 00:10:31.715,3204.325022,3.844682e+11,1.216406e+10
3,2024-04-23 00:15:57.075,3207.880395,3.851852e+11,1.212057e+10
4,2024-04-23 00:20:22.624,3205.428908,3.851852e+11,1.214108e+10
5,2024-04-23 00:25:26.854,3206.680709,3.848231e+11,1.224684e+10
6,2024-04-23 00:30:19.148,3205.104973,3.848231e+11,1.225160e+10
7,2024-04-23 00:36:22.149,3214.098281,3.849843e+11,1.235979e+10
8,2024-04-23 00:40:41.440,3215.832140,3.849843e+11,1.220662e+10
9,2024-04-23 00:45:51.366,3213.808616,3.862583e+11,1.224950e+10


### transform this data into ohlc (open, high, low, close) data for the past day

In [68]:
eth_new = eth_raw.resample('D', on='date').agg({
    'price': ['first', 'max', 'min', 'last', 'mean'],
    'volume_24h': 'last',
    'market_cap': 'last'
}).reset_index()
eth_new.columns = ['date', 'open', 'high', 'low', 'close', 'avg_price', 'volume', 'market_cap']
eth_new['volume_eth'] = eth_new['volume'] / eth_new['avg_price']
eth_new = eth_new[['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']]

eth_new


,date,open,high,low,close,volume,volume_eth,market_cap
0,2024-04-22,3146.652021,3232.067831,3134.798198,3199.76959,1.210134e+10,3.788190e+06,3.844669e+11


## Combine this recent data with what we already have in hopsworks feature store

In [49]:
eth_old = pull_data('eth_ohlc_transformed', 1, 'eth_ohlc_transformed_view', 1)
eth_old

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.27s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,2015-11-15 00:00:00+00:00,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.93685,0.032503,...,0,1,0,0,0,0,0,0,0,1
1225,2015-11-16 00:00:00+00:00,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.02000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2432,2015-11-17 00:00:00+00:00,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,1.00000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
1968,2015-11-18 00:00:00+00:00,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.98275,-0.007377,...,0,1,0,0,0,1,0,0,0,0
859,2015-11-19 00:00:00+00:00,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.94030,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2024-04-17 00:00:00+00:00,3083.0000,3119.0000,2926.0000,2984.0000,1.550000e+10,5093979.0,3.722000e+11,3075.50000,0.029751,...,0,0,0,0,0,1,0,0,0,0
2751,2024-04-18 00:00:00+00:00,2986.0000,3087.0000,2960.0000,3064.0000,1.350000e+10,4453302.0,3.706000e+11,3101.00000,0.011932,...,0,0,0,0,0,0,1,0,0,0
2793,2024-04-19 00:00:00+00:00,3061.0000,3117.0000,2879.0000,3085.0000,1.740000e+10,5675824.0,3.746000e+11,3160.50000,0.023889,...,0,0,0,0,0,0,0,1,0,0
2705,2024-04-20 00:00:00+00:00,3085.0000,3166.0000,3025.0000,3155.0000,8.900000e+09,2884086.0,3.776000e+11,3168.50000,0.004261,...,0,0,0,0,0,0,0,0,1,0


### Combine the raw, untransformed portion of the old date with the new data we just got from CoinGecko

In [50]:
columns_needed = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']

# Convert date columns to datetime objects, in case they are not already
eth_old['date'] = pd.to_datetime(eth_old['date'])
eth_new['date'] = pd.to_datetime(eth_new['date'])

# Remove any timezone information to make the comparison straightforward
eth_old['date'] = eth_old['date'].dt.tz_localize(None)
eth_new['date'] = eth_new['date'].dt.tz_localize(None)

if not any(eth_new['date'].isin(eth_old['date'])):
    # Since eth_new's date isn't found, just concatenate it
    eth_combined = pd.concat([eth_old[columns_needed], eth_new], ignore_index=True)
else:
    # If the date is found, remove the last row from eth_old and add on the new row
    eth_combined = pd.concat([eth_old.iloc[:-1], eth_new], ignore_index=True)

eth_combined

,date,open,high,low,close,volume,volume_eth,market_cap
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07
...,...,...,...,...,...,...,...,...
3058,2024-04-18,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11
3059,2024-04-19,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11
3060,2024-04-20,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11
3061,2024-04-21,3153.000000,3191.000000,3120.000000,3146.000000,8.200000e+09,2.603957e+06,3.867000e+11


### Use this combined raw dataset to regenerate all the response and predictor variables for the past observations, and for the new observation that was just added

In [51]:
def add_response_vars(data):

    df = data.copy()

    df["tmw_avg_high_close"] = (df["high"].shift(-1) + df["close"].shift(-1)) / 2
    df['tmw_percent_increase'] = ((df['tmw_avg_high_close'] - df['close']) / df['tmw_avg_high_close'])

    # Create binary response variable for if tomorrows return percent is positive
    df['tmw_positive_percent_increase_binary'] = (df['tmw_percent_increase'] > 0).astype(int)

    # Create binary response variables for 1-3 percent increases, by .25 steps
    intervals = [1 + i * 0.25 for i in range(int((3 - 1) / 0.25) + 1)]
    for i in intervals:
        threshold = i / 100
        column_name = f"tmw_{str(i).replace('.', '_')}_percent_increase_binary"
        df[column_name] = (df['tmw_percent_increase'] >= threshold).astype(int)

    # Last row doesn't have a value for tomorrow's return, but it was assigned 0's for response variable. Convert them to NA
    columns = [
    'tmw_positive_percent_increase_binary',
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary'
    ]
    
    # Set the last row's specified columns to NaN
    df.loc[df.index[-1], columns] = pd.NA
    return df

def add_predictor_vars(data):

    df = data.copy()

    ###### Predictor variables about the price momentum

    horizons = [2,5,10,25,50,100] 
    predictors = []
    for horizon in horizons:
        ### Exponential Moving Average (EMA) for past horizon
        ema_col = f"ema_{horizon}"
        df[ema_col] = df["close"] / ta.ema(df["close"], length=horizon)
        predictors.append(ema_col)

        ### Relative Strength Index (RSI) for past horizon
        rsi_col = f"rsi_{horizon}"
        df[rsi_col] = ta.rsi(df["close"], length=horizon)
        predictors.append(rsi_col)

        ### Simple Moving Average (SMA) for past horizon
        sma_col = f"sma_{horizon}"
        df[sma_col] = df["close"] / ta.sma(df["close"], length=horizon)
        predictors.append(sma_col)

        ### Sum of the number of days that had a certain percent increase for the past horizon
        columns = [
            # 'tmw_positive_percent_increase_binary',
            'tmw_1_0_percent_increase_binary',
            'tmw_1_25_percent_increase_binary',
            'tmw_1_5_percent_increase_binary',
            'tmw_1_75_percent_increase_binary',
            'tmw_2_0_percent_increase_binary',
            'tmw_2_25_percent_increase_binary',
            'tmw_2_5_percent_increase_binary',
            'tmw_2_75_percent_increase_binary',
            'tmw_3_0_percent_increase_binary'
        ]
        for col in columns:
            percent = col.split('tmw_')[1].split('_percent')[0]
            trend_col = f"last_{horizon}_day_{percent}_percent_increase_count"
            if col in df.columns:
                df[trend_col] = df[col].shift(1).rolling(horizon).sum()
            else:
                print('PREDICTORS NOT ADDED - You must run add_response_vars() first so that predictors can be created based on that info')
                break    
    
    ###### Predictor variables about seasonality/cycles

    month_names = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    for i, month in enumerate(month_names):
        df[month] = (df['date'].dt.month == i).astype(int)

    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    for i, day in enumerate(days):
        df[day] = (df['date'].dt.dayofweek == i).astype(int)

    return df

In [52]:
eth_transformed = add_response_vars(eth_combined)
eth_transformed = add_predictor_vars(eth_transformed)

# Drop the rows at the start that have NA's for all the rolling averages/metrics, but not the last row since it has NA's just for tomorrow's values
eth_transformed = pd.concat([eth_transformed.iloc[:-1].dropna(), eth_transformed.iloc[-1:]])

### take out the last row of eth_old and append to it the last 2 rows of eth_transformed. set the result to the final, updated dataframe, eth_complete 

In [60]:
eth_complete = pd.concat([eth_old.iloc[:-1], eth_transformed.iloc[-2:]], ignore_index=True)

eth_complete


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,1,0,0,0,0,0,0,0,1
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,1,0,0,0,1,0,0,0,0
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3058,2024-04-18,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,0,0,0,0,0,0,1,0,0,0
3059,2024-04-19,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,0,0,0,0,0,0,0,1,0,0
3060,2024-04-20,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3168.500000,0.004261,...,0,0,0,0,0,0,0,0,1,0
3061,2024-04-21,3153.000000,3191.000000,3120.000000,3146.000000,8.200000e+09,2.603957e+06,3.867000e+11,3215.178412,0.021516,...,0,0,0,0,0,0,0,0,0,1


## Save the data back into Hopsworks

In [64]:
# make sure some of the column types are what hopsworks wants
columns = [
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary',
    'tmw_positive_percent_increase_binary'
]
eth_complete[columns] = eth_complete[columns].astype('Int64')

upload_data(eth_complete, 'eth_ohlc_transformed', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 3063/3063 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: eth_ohlc_transformed_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/eth_ohlc_transformed_1_offline_fg_materialization/executions
